In [8]:
import torch
import torch.nn as nn
import os
from mmdet.apis import init_detector, inference_detector, show_result, inference_detector_feats
from mmdet.models import build_detector
import mmcv
from mmcv.runner import load_checkpoint
from mmdet.datasets.pipelines import Compose
from mmcv.parallel import collate, scatter
import numpy as np
import glob
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3"  # specify which GPU(s) to be used
device='cuda:0'


In [2]:
idx = 0 # 0 for faster rcnn, 1 for retinanet 
config_file = ['configs/tbtc_fater_rcnn_voc.py',
               'tbtc_retinanet_voc.py','tbtc_feature_exteactor_faster_rcnn.py',
               'tbtc_feature_exteactor_faster_rcnn.py'
              ][idx]
checkpoint_file = ['checkpoints/faster_rcnn_x101_64x4d_fpn_1x20200324-ba5926a5.pth',
                   'retinanet_x101_64x4d_fpn_1x20200322-53c08bb4.pth'][idx]
# config = mmcv.Config.fromfile(config_file)
# model = build_detector(config.model)
# model = model.to(device)
# model = init_detector(config_file, checkpoint_file,device=device)
# build the model from a config file and a checkpoint file

In [7]:
class LoadImage(object):

    def __call__(self, results):
        if isinstance(results['img'], str):
            results['filename'] = results['img']
        else:
            results['filename'] = None
        img = mmcv.imread(results['img'])
        results['img'] = img
        results['img_shape'] = img.shape
        results['ori_shape'] = img.shape
        return results

def get_img(cfg,img):
    # build the data pipeline
    test_pipeline = [LoadImage()] + cfg.data.test.pipeline[1:]
    test_pipeline = Compose(test_pipeline)
    # H x W x C -> C x H x W and expand an dim
    data = dict(img=img)
    data = test_pipeline(data)
    return data
def get_model(config_file='configs/my.py',
              checkpoint_file='work_dirs/retinanet_x101_64x4d_fpn_1x/latest.pth',
              device='cuda:0'):
    model = init_detector(config_file, checkpoint_file, device=device)
    return model


In [8]:
img = 'demo/tbtc_test.jpg'
config = mmcv.Config.fromfile(config_file)
data = get_img(config,img)
data

{'img_metas': [DataContainer({'filename': 'demo/tbtc_test.jpg', 'ori_shape': (800, 800, 3), 'img_shape': (800, 800, 3), 'pad_shape': (800, 800, 3), 'scale_factor': 1.0, 'flip': False, 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}})],
 'img': [tensor([[[-0.9534, -0.9877, -0.9534,  ...,  0.3823,  0.3652,  0.3823],
           [-0.8335, -0.8164, -0.7822,  ...,  0.3309,  0.3309,  0.3652],
           [-0.7137, -0.6452, -0.6281,  ...,  0.2796,  0.2967,  0.3481],
           ...,
           [-0.1828, -0.0972, -0.1143,  ...,  0.3481,  0.3481,  0.2453],
           [-0.1828, -0.1486, -0.0458,  ...,  0.3138,  0.3138,  0.1254],
           [ 0.0056,  0.0569,  0.2967,  ...,  0.4851,  0.3652,  0.2453]],
  
          [[-0.8277, -0.8627, -0.8277,  ...,  0.4853,  0.4853,  0.5028],
           [-0.7052, -0.6877, -0.6527,  ...,  0.4328,  0.4503,  0.4853],
           [-0.5826, -0.5126, -0.4951,  ...,  0.3803, 

In [9]:
img_metas = [ i.data for i in data['img_metas'] ]
print(img_metas)

[{'filename': 'demo/tbtc_test.jpg', 'ori_shape': (800, 800, 3), 'img_shape': (800, 800, 3), 'pad_shape': (800, 800, 3), 'scale_factor': 1.0, 'flip': False, 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}]


In [3]:
model = init_detector(config_file, checkpoint_file,device=device)

In [4]:
img = 'demo/tbtc_test.jpg'
result,roi_feats = inference_detector(model,img)

In [6]:
roi_feats.shape

torch.Size([1000, 256, 7, 7])

In [20]:
x[2].shape

torch.Size([1, 256, 50, 50])

In [33]:
img_ = 'demo/tbtv_test_small.jpg'

In [34]:
result,roi_feats = inference_detector(model,img_)

In [35]:
roi_feats.shape

torch.Size([1000, 256, 7, 7])

In [9]:
data_path = glob.glob('data/coco/images/*')

In [10]:
data_path

['data/coco/images/train2017',
 'data/coco/images/val2017',
 'data/coco/images/test2017']

In [11]:
img_paths = []
for path in data_path[:10]:
    img_paths.extend(glob.glob(path + '/*'))

In [12]:
len(img_paths)

248166

In [13]:
import mmcv

In [28]:
for i in img_paths[:10000]:
    img = mmcv.imread(i)
    if img.shape != (800, 800, 3):
        print(i)
        print(img.shape)

data/coco/images/train2017/070977_1.jpg
(750, 750, 3)
data/coco/images/train2017/075926_1.jpg
(750, 750, 3)
data/coco/images/train2017/119948_4.jpg
(600, 400, 3)
data/coco/images/train2017/098842_3.jpg
(750, 750, 3)
data/coco/images/train2017/105110_4.jpg
(800, 600, 3)
data/coco/images/train2017/073653_4.jpg
(750, 750, 3)
data/coco/images/train2017/084132_1.jpg
(750, 750, 3)
data/coco/images/train2017/084368_0.jpg
(800, 750, 3)
data/coco/images/train2017/100606_2.jpg
(800, 600, 3)
data/coco/images/train2017/082845_0.jpg
(750, 750, 3)
data/coco/images/train2017/035619_4.jpg
(800, 750, 3)
data/coco/images/train2017/097999_4.jpg
(800, 600, 3)
data/coco/images/train2017/010483_1.jpg
(800, 750, 3)
data/coco/images/train2017/117068_1.jpg
(750, 750, 3)
data/coco/images/train2017/113431_1.jpg
(750, 750, 3)
data/coco/images/train2017/047968_0.jpg
(750, 750, 3)
data/coco/images/train2017/118188_0.jpg
(800, 600, 3)
data/coco/images/train2017/117206_0.jpg
(800, 706, 3)
data/coco/images/train2017/0

KeyboardInterrupt: 

In [23]:
img.shape == (800, 800, 3)

True

In [49]:
shape_dict = {}

In [51]:
for task in mmcv.track_iter_progress(img_paths[:1000]):
    img = mmcv.imread(task)
    shape = str(img.shape)
    if shape in shape_dict.keys():
        shape_dict[shape]+=1
    else:
        shape_dict[shape]=1
        

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1000/1000, 80.8 task/s, elapsed: 12s, ETA:     0s


In [52]:
shape_dict

{'(800, 800, 3)': 942,
 '(750, 750, 3)': 28,
 '(600, 400, 3)': 1,
 '(800, 600, 3)': 12,
 '(800, 750, 3)': 5,
 '(800, 706, 3)': 1,
 '(800, 599, 3)': 1,
 '(800, 553, 3)': 1,
 '(736, 742, 3)': 1,
 '(700, 700, 3)': 2,
 '(800, 742, 3)': 1,
 '(800, 574, 3)': 1,
 '(800, 533, 3)': 1,
 '(500, 750, 3)': 1,
 '(768, 768, 3)': 1,
 '(706, 600, 3)': 1}

In [9]:
def fun(*args, **kwargs):
#     print('args=', args)
    print('kwargs=', kwargs)

In [10]:
fun(**data)

kwargs= {'img_metas': [DataContainer({'filename': 'demo/tbtc_test.jpg', 'ori_shape': (800, 800, 3), 'img_shape': (800, 800, 3), 'pad_shape': (800, 800, 3), 'scale_factor': 1.0, 'flip': False, 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}})], 'img': [tensor([[[-0.9534, -0.9877, -0.9534,  ...,  0.3823,  0.3652,  0.3823],
         [-0.8335, -0.8164, -0.7822,  ...,  0.3309,  0.3309,  0.3652],
         [-0.7137, -0.6452, -0.6281,  ...,  0.2796,  0.2967,  0.3481],
         ...,
         [-0.1828, -0.0972, -0.1143,  ...,  0.3481,  0.3481,  0.2453],
         [-0.1828, -0.1486, -0.0458,  ...,  0.3138,  0.3138,  0.1254],
         [ 0.0056,  0.0569,  0.2967,  ...,  0.4851,  0.3652,  0.2453]],

        [[-0.8277, -0.8627, -0.8277,  ...,  0.4853,  0.4853,  0.5028],
         [-0.7052, -0.6877, -0.6527,  ...,  0.4328,  0.4503,  0.4853],
         [-0.5826, -0.5126, -0.4951,  ...,  0.3803,  0.4153,  0.4